In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
from PIL import Image
import glob
import matplotlib.pyplot as plt
import numpy as np
import cv2
directory='/content/drive/MyDrive/Female_and_male_eyes/femaleeyes'
image_list_females = []
image_list_males = []
for filename in glob.glob('/content/drive/MyDrive/Female_and_male_eyes/femaleeyes/*.jpg'):
  im = cv2.imread(filename)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  image_list_females.append(im) 
for filename in glob.glob('/content/drive/MyDrive/Female_and_male_eyes/maleeyes/*.jpg'):
  im = cv2.imread(filename)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  image_list_males.append(im) 

In [ ]:
import csv
import pandas as pd
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.filters import prewitt_h,prewitt_v


data_to_append_females=[]
data_to_append_males=[]




for img in image_list_females:
  imm = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  imm=cv2.resize(imm,(50,50))
  equalized_img = cv2.equalizeHist(imm)
  # plt.imshow(equalized_img,cmap='gray')

  mean_img=equalized_img.mean()
  median_img=np.median(equalized_img)
  std_img=equalized_img.std()
  var_img=equalized_img.var()
  skew_img=(3*(mean_img-median_img))/std_img
  entropy_img=entropy(equalized_img, disk(5)).mean()
  prewitt_h_img=prewitt_h(equalized_img).mean()
  prewitt_v_img=prewitt_v(equalized_img).mean()
  sobelx_img = cv2.Sobel(equalized_img,cv2.CV_64F,1,0,ksize=5).mean()
  sobely_img = cv2.Sobel(equalized_img,cv2.CV_64F,0,1,ksize=5).mean()

  app=[mean_img,median_img,std_img,var_img,skew_img,entropy_img,prewitt_h_img,prewitt_v_img,sobelx_img,sobely_img,'female']
  data_to_append_females.append(app)

for img in image_list_males:
  imm = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  imm=cv2.resize(imm,(50,50))
  equalized_img = cv2.equalizeHist(imm)
  # plt.imshow(equalized_img,cmap='gray')

  mean_img=equalized_img.mean()
  median_img=np.median(equalized_img)
  std_img=equalized_img.std()
  var_img=equalized_img.var()
  skew_img=(3*(mean_img-median_img))/std_img
  entropy_img=entropy(equalized_img, disk(5)).mean()
  prewitt_h_img=prewitt_h(equalized_img).mean()
  prewitt_v_img=prewitt_v(equalized_img).mean()
  sobelx_img = cv2.Sobel(equalized_img,cv2.CV_64F,1,0,ksize=5).mean()
  sobely_img = cv2.Sobel(equalized_img,cv2.CV_64F,0,1,ksize=5).mean()

  app=[mean_img,median_img,std_img,var_img,skew_img,entropy_img,prewitt_h_img,prewitt_v_img,sobelx_img,sobely_img,'male']
  data_to_append_males.append(app)

data_to_append_females=np.array(data_to_append_females)
data_to_append_males=np.array(data_to_append_males)

file= open('Female_and_Male_eyes.csv','a',newline='')
writer=csv.writer(file)
writer.writerows(data_to_append_females)
writer.writerows(data_to_append_males)

file.close()

In [ ]:
#Apply Decision Tree classifier
df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['label'])
y = df['label']
#------------------------------------------
from pickle import decode_long
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.606323083585968

In [ ]:
#using only five features

df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
#-----------------------------------------
from pickle import decode_long
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.5158077089649199

In [ ]:
#using only two features

df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['std','variance','skew','entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
#---------------------------------------------
from pickle import decode_long
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.49545257687310523

In [ ]:
# Applying Random Forest classifier
df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['label'])
y = df['label']
#---------------------------------------
from pickle import decode_long
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.7033347769597228

In [ ]:
#using only five features

df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
from pickle import decode_long
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.5651797314854916

In [ ]:
#using only two features

df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['std','variance','skew','entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
from pickle import decode_long
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.5067128627111304

In [ ]:
#applying LogisticRegression classifier
df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['label'])
y = df['label']
from pickle import decode_long
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=LogisticRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.6543958423559982

In [ ]:
#using only five features
df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
from pickle import decode_long
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.5495885664789952

In [ ]:
#using only two features

df = pd.read_csv('Female_and_Male_eyes.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = df.drop(columns=['std','variance','skew','entropy','prewitt_x','prewitt_y','sobel_x','sobel_y','label'])
y = df['label']
from pickle import decode_long
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.utils import to_categorical
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=0.2)

model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

score=accuracy_score(y_test,predictions)
score

0.5071459506279775